In [1]:
import math
import numpy as np
import numpy.random as npr
from numpy import linalg
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset
import torch.optim as optim
import time
from math import *
torch.cuda.set_device(1)

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
class ResNet(nn.Module):
    def __init__(self,dim, m,o):
        super(ResNet, self).__init__()
        self.Ix = torch.zeros([dim,m]).cuda()
        self.Ix[0,0] = 1
        self.Ix[1,1] = 1
        self.fc1 = nn.Linear(dim, m)
        self.fc2 = nn.Linear(m, m)
        
        self.fc3 = nn.Linear(m, m)
        self.fc4 = nn.Linear(m, m)

        
        self.outlayer = nn.Linear(m, o)

    def forward(self, x):
        s = x@self.Ix
        y = self.fc1(x)
        y = F.relu(y**deg)
        y = self.fc2(y)
        y = F.relu(y**deg)
        y = y+s
        
        s=y
        y = self.fc3(y)
        y = F.relu(y**deg)
        y = self.fc4(y)
        y = F.relu(y**deg)
        y = y+s
        
        
        output = self.outlayer(y)
        return output
deg = 2

In [4]:
def sol_exact(x):
    x_tem = torch.cos(pi*x[:,1:dim_set])
    sol = torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return sol * x[:,0:1]*(1-x[:,0:1])

In [5]:
def sol_exact_H1(x):
    sol = torch.zeros(x.size()[0],dim_set+1).cuda()
    sol[:,0:1] =  x[:,0:1]*(1-x[:,0:1])*torch.sum(torch.cos(pi*x[:,1:dim_set]),dim=1).reshape([x.size()[0],1])
    sol[:,1:2] = (1-2*x[:,0:1])*torch.sum(torch.cos(pi*x[:,1:dim_set]),dim=1).reshape([x.size()[0],1])
    sol[:,2:dim_set+1] = -pi*x[:,0:1]*(1-x[:,0:1])*torch.sin(pi*x[:,1:dim_set])
    return sol

In [6]:
def right(x):
    x_tem = torch.cos(pi*x[:,1:dim_set])
    f_tem = (-2-pi**2*x[:,0:1]*(1-x[:,0:1]))*torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return f_tem

In [7]:
def BCDataGen(BatchSize):
    bound = torch.rand(BatchSize,1)
    perbd = int(BatchSize/2)
    bound[0:perbd,0]=1.
    bound[perbd:2*perbd,0]=0.
    return bound

In [8]:
dim_set = 16
Batch_size = 50000
def DRM1():
    x = torch.rand(Batch_size,dim_set).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    ux = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    loss1 = torch.sum((ux-du)**2)/Batch_size
    return loss1

In [9]:
def DRM2():
    x = torch.rand(Batch_size,dim_set).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    uxx = torch.zeros(Batch_size,dim_set).cuda()
    for i in range(dim_set):
        ux_tem = du[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        uxx[:,i] = uxx_tem[:,i]
    loss2 = torch.sum((torch.sum(uxx,dim=1).reshape([x.size()[0],1])  - right(x))**2)/Batch_size
    return loss2

In [10]:
def model_p(x):
    multi = x*(1-x)
    multi[:,0] = 1
    return model_p1(x)*multi

In [11]:
def model_u(x):
    return (1-x[:,0:1])*x[:,0:1]*model_u1(x) 

In [12]:
model_u1 = ResNet(dim_set,24,1)
model_p1 = ResNet(dim_set,24,dim_set)

device=torch.device("cuda:1" )
model_u1.to(device)
model_p1.to(device)

ResNet(
  (fc1): Linear(in_features=16, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=24, bias=True)
  (fc4): Linear(in_features=24, out_features=24, bias=True)
  (outlayer): Linear(in_features=24, out_features=16, bias=True)
)

In [13]:
notes = torch.rand(10000,dim_set).cuda()
exact = sol_exact_H1(notes)
u_L2 = sqrt(torch.sum((exact[:,0])**2)/10000)
Du_L2 = sqrt(torch.sum(exact[:,1:dim_set+1]**2)/10000)
def loss_error():
    notes = torch.rand(10000,dim_set).cuda()
    predict = torch.zeros(10000,2*dim_set+2).cuda()
    predict[:,0]= model_u(notes).reshape([1,notes.size()[0]])
    predict[:,1:dim_set+1]= model_p(notes)
    exact = sol_exact_H1(notes)
    value1 = sqrt(torch.sum((predict[:,0] - exact[:,0])**2)/10000)/u_L2
    value2 = sqrt(torch.sum((predict[:,1:dim_set+1] - exact[:,1:dim_set+1])**2)/10000)/Du_L2
    return value1,value2

In [14]:
traintime = 50000
error_save=np.zeros([2,traintime])
optimizer = optim.Adam([
                {'params': model_u1.parameters()},
                {'params': model_p1.parameters()},
            ])
time_start=time.time()
for i in range(traintime):
    optimizer.zero_grad()
    losses1 = DRM1()
    losses1.backward()
    optimizer.step() 
    optimizer.zero_grad()
    losses2 = DRM2()
    losses2.backward()
    optimizer.step() 
    error1,error2 = loss_error()
    error_save[0,i]=float(error1)
    error_save[1,i]=float(error2)
    #scheduler.step()
    if i%500==1:
        print("i= ",i)
        print("error =",error1)
        print("loss1 =",losses1.detach())
        print("loss2 =",losses2.detach())
        np.save("error_save_LDG_NBC_16D_v2.npy", error_save)
np.save("error_save_LDG_NBC_16D_v2.npy", error_save)
time_end=time.time()
print('time cost',time_end-time_start,'s')

i=  1
error = 1.0045090007891433
loss1 = tensor(0.0441, device='cuda:1')
loss2 = tensor(104.4639, device='cuda:1')
i=  501
error = 0.17784066943112814
loss1 = tensor(1.5755, device='cuda:1')
loss2 = tensor(2.1340, device='cuda:1')
i=  1001
error = 0.14914972892366554
loss1 = tensor(0.5205, device='cuda:1')
loss2 = tensor(1.5739, device='cuda:1')
i=  1501
error = 0.13966713546045254
loss1 = tensor(0.3286, device='cuda:1')
loss2 = tensor(1.4181, device='cuda:1')
i=  2001
error = 0.12138413906010159
loss1 = tensor(0.2808, device='cuda:1')
loss2 = tensor(1.4530, device='cuda:1')
i=  2501
error = 0.1161072298973604
loss1 = tensor(0.2687, device='cuda:1')
loss2 = tensor(1.2818, device='cuda:1')
i=  3001
error = 0.10912333834049756
loss1 = tensor(0.2697, device='cuda:1')
loss2 = tensor(1.2112, device='cuda:1')
i=  3501
error = 0.10146187032903584
loss1 = tensor(0.2790, device='cuda:1')
loss2 = tensor(1.1470, device='cuda:1')
i=  4001
error = 0.10085786505756245
loss1 = tensor(0.2788, device='

i=  35001
error = 0.018505616950575458
loss1 = tensor(0.0717, device='cuda:1')
loss2 = tensor(0.0107, device='cuda:1')
i=  35501
error = 0.012528659751516001
loss1 = tensor(0.0708, device='cuda:1')
loss2 = tensor(0.0124, device='cuda:1')
i=  36001
error = 0.020332319640133486
loss1 = tensor(0.0719, device='cuda:1')
loss2 = tensor(0.0127, device='cuda:1')
i=  36501
error = 0.012286540062784476
loss1 = tensor(0.0713, device='cuda:1')
loss2 = tensor(0.0121, device='cuda:1')
i=  37001
error = 0.0317743876100114
loss1 = tensor(0.0725, device='cuda:1')
loss2 = tensor(0.0118, device='cuda:1')
i=  37501
error = 0.011580677883197795
loss1 = tensor(0.0702, device='cuda:1')
loss2 = tensor(0.0154, device='cuda:1')
i=  38001
error = 0.01316459056253818
loss1 = tensor(0.0706, device='cuda:1')
loss2 = tensor(0.0100, device='cuda:1')
i=  38501
error = 0.011030678289388729
loss1 = tensor(0.0705, device='cuda:1')
loss2 = tensor(0.0144, device='cuda:1')
i=  39001
error = 0.03843484761896063
loss1 = tenso

In [15]:
import matplotlib.pyplot as plt

In [16]:
index = np.arange(0,50000,10)
plt.plot(np.log10(error_save[index]))

IndexError: index 10 is out of bounds for axis 0 with size 2

In [ ]:
def model_u(x):
    return sol_exact_H1(x)[:,0:1]
def model_p(x):
    return sol_exact_H1(x)[:,1:dim_set+1]